In [4]:
import pandas as pd
from rapidfuzz import process, fuzz
import difflib
import re
import unicodedata

In [5]:
smm = pd.read_csv('smm.csv')


In [42]:

TIPOS_VIA = ['rua ', 'largo ', 'travessa ', 'avenida ', 'praça ', 'beco ', 'calçada ', 'estrada ']
ARTIGOS = ['do ', 'da ', 'dos ', 'das ', 'de ', "d'", '']

def remover_acentos(texto):
    """Remove acentos de um texto"""
    if pd.isna(texto) or not isinstance(texto, str):
        return ""
    return ''.join(c for c in unicodedata.normalize('NFD', texto) 
                   if unicodedata.category(c) != 'Mn')

def normalizar_rua(rua):
    if pd.isna(rua) or not isinstance(rua, str):
        return ""
    return re.sub(r'\s+', ' ', rua.lower().strip())

def normalizar_rua_para_matching(rua):
    """Normalização robusta específica para fuzzy matching"""
    if pd.isna(rua) or not isinstance(rua, str):
        return ""
    
    # Converter para lowercase e remover espaços extras
    rua = re.sub(r'\s+', ' ', rua.lower().strip())
    
    # Remover acentos
    rua = remover_acentos(rua)
    
    # Tratar preposições que podem estar ausentes ou extras
    # Remove "de" quando está sozinho entre palavras
    rua = re.sub(r'\bde\s+(?=\w)', '', rua)
    
    # Limpar espaços duplicados resultantes
    rua = re.sub(r'\s+', ' ', rua).strip()
    
    return rua

def extrair_tipo_via(rua):
    rua_norm = normalizar_rua(rua)
    for tipo in TIPOS_VIA:
        if rua_norm.startswith(tipo):
            return {tipo.strip()}
    return set()

def nome_base_sem_artigos(rua):
    rua_norm = normalizar_rua(rua)
    for tipo in TIPOS_VIA:
        if rua_norm.startswith(tipo):
            rua_norm = rua_norm[len(tipo):].strip()
            break
    for artigo in ARTIGOS:
        if rua_norm.startswith(artigo):
            rua_norm = rua_norm[len(artigo):].strip()
            break
    return rua_norm

def detectar_dados_malformados(rua):
    if pd.isna(rua) or not isinstance(rua, str):
        return True
    rua_limpa = rua.strip().lower()
    problemas = [
        len(rua_limpa) < 3,
        rua_limpa in ['rua', 'avenida', 'travessa', 'largo', 'praça'],
        rua_limpa.startswith('rua rua'),
        rua_limpa.startswith('avenida avenida'),
        rua_limpa.count(' ') == 0 and len(rua_limpa) < 5,
        re.match(r'^[0-9\s\-_]+$', rua_limpa),
        rua_limpa in ['nan', 'null', 'none', ''],
    ]
    return any(problemas)

def extrair_palavras_chave(rua):
    stop_words = {'da', 'do', 'de', 'das', 'dos', 'e', 'em', 'na', 'no', 'para', 'por', 'com', 'sem'}
    palavras = set(normalizar_rua(rua).split())
    return palavras - stop_words

def calcular_similaridade_contextual(rua1, rua2):
    palavras1 = extrair_palavras_chave(rua1)
    palavras2 = extrair_palavras_chave(rua2)
    if not palavras1 or not palavras2:
        return 0
    intersecao = palavras1.intersection(palavras2)
    uniao = palavras1.union(palavras2)
    return len(intersecao) / len(uniao) if uniao else 0

# --- Correspondência exata robusta ---

def verificar_correspondencia_exata_melhorada(rua_alojamento, ruas_oficiais):
    """Busca correspondência exata, priorizando tipo de via e ignorando artigos"""
    rua_alojamento_norm = normalizar_rua(rua_alojamento)
    tipo_aloj = list(extrair_tipo_via(rua_alojamento_norm))
    nome_base_aloj = nome_base_sem_artigos(rua_alojamento_norm)

    # 1. Correspondência exata total (com tipo de via e artigos)
    for rua in ruas_oficiais:
        if normalizar_rua(rua) == rua_alojamento_norm:
            return rua, 100  # match perfeito

    # 2. Correspondência exata de tipo de via e nome base (ignorando artigos)
    for rua in ruas_oficiais:
        rua_norm = normalizar_rua(rua)
        tipo_oficial = list(extrair_tipo_via(rua_norm))
        nome_base_oficial = nome_base_sem_artigos(rua_norm)
        if tipo_oficial == tipo_aloj and nome_base_oficial == nome_base_aloj:
            return rua, 98  # match de tipo e nome base

    # 3. NOVA VERIFICAÇÃO: Correspondência com normalização para matching
    rua_aloj_matching = normalizar_rua_para_matching(rua_alojamento)
    for rua in ruas_oficiais:
        rua_oficial_matching = normalizar_rua_para_matching(rua)
        if rua_aloj_matching == rua_oficial_matching:
            return rua, 97  # match normalizado

    # 3. NÃO aceite correspondência de tipo diferente, mesmo que o nome base seja igual!
    return None, 0

def existe_tipo_via_exato(rua_alojamento, ruas_oficiais):
    tipo_aloj = list(extrair_tipo_via(rua_alojamento))
    nome_base_aloj = nome_base_sem_artigos(rua_alojamento)
    for rua in ruas_oficiais:
        if pd.isna(rua):
            continue
        tipo_oficial = list(extrair_tipo_via(rua))
        nome_base_oficial = nome_base_sem_artigos(rua)
        if tipo_oficial == tipo_aloj and nome_base_oficial == nome_base_aloj:
            return True
    return False

# --- Validação final ---

def validar_correspondencia_melhorada(rua_original, rua_sugerida, score_fuzzy, ruas_oficiais=None):
    if detectar_dados_malformados(rua_original):
        return False, "dados_malformados"
    
    # MELHORIA: Verificar se existe correspondência de nome mais forte que tipo de via
    if ruas_oficiais is not None:
        nome_base_original = nome_base_sem_artigos(rua_original)
        nome_base_sugerida = nome_base_sem_artigos(rua_sugerida)
        
        # Se o nome base é muito similar, verificar se há alternativa com nome mais próximo
        if score_fuzzy >= 90 and len(nome_base_original) > 5:
            # Procurar por ruas com o mesmo nome base, independente do tipo de via
            for rua_alternativa in ruas_oficiais:
                nome_base_alt = nome_base_sem_artigos(rua_alternativa)
                # Se encontrar nome base idêntico ou muito similar
                if (nome_base_original == nome_base_alt or 
                    fuzz.ratio(nome_base_original, nome_base_alt) >= 95):
                    # Se a alternativa tem nome melhor, prefira ela
                    if fuzz.ratio(nome_base_original, nome_base_alt) > fuzz.ratio(nome_base_original, nome_base_sugerida):
                        return False, "existe_nome_mais_proximo"
        
        # Relaxar a regra de tipo de via quando o score é muito alto
        if existe_tipo_via_exato(rua_original, ruas_oficiais) and score_fuzzy < 95:
            tipo1 = extrair_tipo_via(rua_original)
            tipo2 = extrair_tipo_via(rua_sugerida)
            if tipo1 != tipo2:
                return False, "tipo_via_exato_disponivel"
    
    if score_fuzzy < 80:
        return False, "score_muito_baixo"
    rua1_norm = normalizar_rua(rua_original)
    rua2_norm = normalizar_rua(rua_sugerida)
    palavras_origem = extrair_palavras_chave(rua_original)
    palavras_destino = extrair_palavras_chave(rua_sugerida)
    nome_origem_sem_via = rua1_norm
    nome_destino_sem_via = rua2_norm
    for tipo in TIPOS_VIA:
        if nome_origem_sem_via.startswith(tipo):
            nome_origem_sem_via = nome_origem_sem_via[len(tipo):].strip()
        if nome_destino_sem_via.startswith(tipo):
            nome_destino_sem_via = nome_destino_sem_via[len(tipo):].strip()
    if (nome_destino_sem_via in nome_origem_sem_via and 
        nome_destino_sem_via != nome_origem_sem_via and
        len(nome_destino_sem_via) < len(nome_origem_sem_via) * 0.7):
        return False, "nome_parcial_quando_completo_disponivel"
    if len(palavras_origem) <= 2:
        nome_principal = None
        for palavra in palavras_origem:
            if palavra not in ['rua', 'avenida', 'travessa', 'largo', 'praça', 'beco']:
                nome_principal = palavra
                break
        if nome_principal:
            if nome_principal not in rua2_norm:
                return False, "nome_principal_ausente"
            palavras_extras = palavras_destino - palavras_origem
            palavras_aceitaveis = {'de', 'do', 'da', 'dos', 'das', 'são', 'santa', 'santo', 'cruz'}
            palavras_extras_relevantes = palavras_extras - palavras_aceitaveis
            limite_palavras_extras = 3 if score_fuzzy >= 96 else 2
            if len(palavras_extras_relevantes) > limite_palavras_extras:
                return False, "muitas_palavras_extras"
    sim_contextual = calcular_similaridade_contextual(rua_original, rua_sugerida)
    min_sim_contextual = 0.4 if len(palavras_origem) <= 2 else 0.3
    if score_fuzzy >= 96:
        min_sim_contextual *= 0.6
    if sim_contextual < min_sim_contextual:
        return False, "contexto_muito_diferente"
    seq_ratio = difflib.SequenceMatcher(None, rua1_norm, rua2_norm).ratio()
    min_seq_ratio = 0.25 if len(palavras_origem) <= 2 else 0.2
    if seq_ratio < min_seq_ratio:
        return False, "sequencia_muito_diferente"
    tipos_via1 = extrair_tipo_via(rua_original)
    tipos_via2 = extrair_tipo_via(rua_sugerida)
    if tipos_via1 and tipos_via2 and not tipos_via1.intersection(tipos_via2):
        tipos_compativeis = {
            'rua': {'travessa', 'beco', 'calçada'},
            'travessa': {'rua', 'beco'},
            'largo': {'praça'},
            'praça': {'largo'}
        }
        compativel = False
        for tipo1 in tipos_via1:
            if tipo1 in tipos_compativeis and tipos_via2.intersection(tipos_compativeis[tipo1]):
                compativel = True
                break
        if not compativel:
            return False, "tipo_via_incompativel"
    diff_tamanho = abs(len(rua1_norm) - len(rua2_norm))
    if diff_tamanho > len(rua1_norm) * 2.0:
        return False, "diferenca_tamanho_excessiva"
    return True, "aprovada"

# --- Matching principal ---

def comparacao_fuzzy_melhorada(rua_alojamento, ruas_oficiais):
    if detectar_dados_malformados(rua_alojamento):
        return "precisa_analise_manual", 0, 'dados_malformados'
    rua_norm = normalizar_rua(rua_alojamento)
    casos_genéricos = {
        'rua rua', 'avenida avenida', 'largo largo', 'praça praça', 'beco beco', 'travessa travessa',
        'rua', 'avenida', 'praça', 'travessa', 'beco', 'largo'
    }
    if rua_norm in casos_genéricos:
        return "precisa_analise_manual", 0, 'generico_ou_duplicado'
    match_exato, score_exato = verificar_correspondencia_exata_melhorada(rua_alojamento, ruas_oficiais)
    if match_exato:
        valida, motivo = validar_correspondencia_melhorada(rua_alojamento, match_exato, score_exato, ruas_oficiais)
        if valida:
            return match_exato, score_exato, 'correspondencia_exata_melhorada'
    
    # NOVA LÓGICA: Buscar primeiro por nome base idêntico, independente do tipo de via
    nome_base_alojamento = nome_base_sem_artigos(rua_alojamento)
    candidatos_nome_identico = []
    
    for rua_oficial in ruas_oficiais:
        nome_base_oficial = nome_base_sem_artigos(rua_oficial)
        if nome_base_alojamento == nome_base_oficial:
            # Nome base idêntico - calcular score
            score = fuzz.token_sort_ratio(rua_alojamento, rua_oficial)
            candidatos_nome_identico.append((rua_oficial, score, 'nome_base_identico'))
    
    # Se encontrou candidatos com nome base idêntico, priorizar o melhor
    if candidatos_nome_identico:
        melhor_nome_identico = max(candidatos_nome_identico, key=lambda x: x[1])
        rua_match, score_match, metodo = melhor_nome_identico
        
        # Validar o candidato
        valida, motivo = validar_correspondencia_melhorada(rua_alojamento, rua_match, score_match, ruas_oficiais)
        if valida:
            return rua_match, score_match, metodo
    
    # Fuzzy matching com normalização melhorada (lógica original)
    rua_aloj_normalizada = normalizar_rua_para_matching(rua_alojamento)
    
    # Criar mapeamento das ruas normalizadas para as originais
    ruas_normalizadas = []
    mapa_ruas = {}
    
    for rua_oficial in ruas_oficiais:
        rua_oficial_norm = normalizar_rua_para_matching(rua_oficial)
        ruas_normalizadas.append(rua_oficial_norm)
        mapa_ruas[rua_oficial_norm] = rua_oficial
    
    algoritmos = [
        (fuzz.token_sort_ratio, 'token_sort'),
        (fuzz.token_set_ratio, 'token_set'),
        (fuzz.partial_ratio, 'partial'),
        (fuzz.ratio, 'ratio')
    ]
    todos_resultados = []
    for scorer, nome in algoritmos:
        try:
            # Fazer matching com as versões normalizadas
            match_norm, score, _ = process.extractOne(
                rua_aloj_normalizada, 
                ruas_normalizadas, 
                scorer=scorer
            )
            if match_norm and score >= 80:
                # Recuperar a rua original
                rua_original = mapa_ruas[match_norm]
                todos_resultados.append((rua_original, score, nome))
        except:
            continue
    if not todos_resultados:
        return None, 0, 'sem_correspondencia_fuzzy'
    candidatos_validos = []
    for match, score, metodo in todos_resultados:
        valida, motivo = validar_correspondencia_melhorada(rua_alojamento, match, score, ruas_oficiais)
        if valida:
            candidatos_validos.append((match, score, metodo, motivo))
    if not candidatos_validos:
        return None, 0, 'validacao_falhou'
    melhor = max(candidatos_validos, key=lambda x: x[1])
    return melhor[0], melhor[1], melhor[2]

# --- Carregamento dos dados ---

df_ruas = pd.read_csv('padraoruas.csv')
df_alojamentos = pd.read_csv('smm.csv')

def contem_santo_antonio(freguesias_str):
    if pd.isna(freguesias_str):
        return False
    return 'santo antónio' in str(freguesias_str).lower()

df_ruas_smm = df_ruas[df_ruas['FREGUESIAS'].apply(contem_santo_antonio)].copy()
ruas_oficiais_smm = df_ruas_smm['rua_padronizada'].dropna().unique()

alojamentos_smm = df_alojamentos[df_alojamentos['Freguesia'] == 'Santo António']
ruas_alojamentos_smm = alojamentos_smm['RuaPadronizada'].dropna().unique()
ruas_alojamentos_validas = [rua for rua in ruas_alojamentos_smm if not detectar_dados_malformados(rua)]

print(f"Total de ruas de alojamento válidas: {len(ruas_alojamentos_validas)}")
print(f"Total de ruas oficiais: {len(ruas_oficiais_smm)}")

# --- Processamento principal e exibição ---

resultados = []
for rua_alojamento in ruas_alojamentos_validas:
    match, score, metodo = comparacao_fuzzy_melhorada(rua_alojamento, ruas_oficiais_smm)
    if match:
        num_alojamentos = len(alojamentos_smm[alojamentos_smm['RuaPadronizada'] == rua_alojamento])
        rua_info = df_ruas_smm[df_ruas_smm['rua_padronizada'] == match]
        freguesia = rua_info.iloc[0]['FREGUESIAS'] if not rua_info.empty else "Não encontrada"
        resultados.append({
            'rua_alojamento': rua_alojamento,
            'rua_oficial': match,
            'score_fuzzy': score,
            'metodo': metodo,
            'num_alojamentos': num_alojamentos,
            'freguesia_oficial': freguesia,
            'similaridade_contextual': calcular_similaridade_contextual(rua_alojamento, match)
        })

print(f"Total de correspondências encontradas: {len(resultados)}\n")

def classificar_sugestao(score, similaridade_contextual, metodo):
    if metodo == 'correspondencia_exata_melhorada' and score >= 97:
        return 'done'  
    if score >= 98:
        return 'done'
    
    if score >= 95 and similaridade_contextual >= 0.4:  # era 0.7
        return 'done'
    elif score >= 90 and similaridade_contextual >= 0.4:  # era 0.5
        return 'done'
    elif score >= 85 and similaridade_contextual >= 0.6:  # novo critério
        return 'done'
    elif score >= 80:
        return 'precisa_analise_manual'
    else:
        return 'precisa_analise_manual'

# Results
resultados_done = []
resultados_analise = []
for r in resultados:
    sugestao = classificar_sugestao(r['score_fuzzy'], r['similaridade_contextual'], r['metodo'])
    if sugestao == 'done':
        resultados_done.append(r)
    else:
        resultados_analise.append(r)

print('Resultados' )
for r in resultados_done:
    indicadores = []
    if r['score_fuzzy'] >= 95:
        indicadores.append("")
    elif r['score_fuzzy'] >= 90:
        indicadores.append("")
    elif r['score_fuzzy'] >= 85:
        indicadores.append("")
    else:
        indicadores.append("")
    if r['similaridade_contextual'] >= 0.7:
        indicadores.append("")
    elif r['similaridade_contextual'] >= 0.5:
        indicadores.append("")
    else:
        indicadores.append("❓")
    print(f"{' '.join(indicadores)} '{r['rua_alojamento']}'")
    print(f"    → '{r['rua_oficial']}'")
    print(f"    SCORE: {r['score_fuzzy']:.1f}% | SIMILARIDADE: {r['similaridade_contextual']:.3f} | Método: {r['metodo']}")
    print(f"    ALOJAMENTOS: {r['num_alojamentos']} |  {r['freguesia_oficial']}")
    print()

print(' PRECISA DE ANÁLISE MANUAL ' )
for r in resultados_analise:
    indicadores = []
    if r['score_fuzzy'] >= 95:
        indicadores.append("")
    elif r['score_fuzzy'] >= 90:
        indicadores.append("")
    elif r['score_fuzzy'] >= 85:
        indicadores.append("")
    else:
        indicadores.append("")
    if r['similaridade_contextual'] >= 0.7:
        indicadores.append("")
    elif r['similaridade_contextual'] >= 0.5:
        indicadores.append("")
    else:
        indicadores.append("")
    print(f"{' '.join(indicadores)} '{r['rua_alojamento']}'")
    print(f"    → '{r['rua_oficial']}'")
    print(f"    SCORE: {r['score_fuzzy']:.1f}% | SIMILARIDADE: {r['similaridade_contextual']:.3f} | Método: {r['metodo']}")
    print(f"    ALOJAMENTOS: {r['num_alojamentos']} |  {r['freguesia_oficial']}")
    print()

Total de ruas de alojamento válidas: 155
Total de ruas oficiais: 133
Total de correspondências encontradas: 144

Resultados
  'rua santo antónio da glória'
    → 'rua de santo antónio da glória'
    SCORE: 98.0% | SIMILARIDADE: 1.000 | Método: correspondencia_exata_melhorada
    ALOJAMENTOS: 32 |  Santo António (Nova Freguesia)

  'travessa do noronha'
    → 'travessa do noronha'
    SCORE: 100.0% | SIMILARIDADE: 1.000 | Método: correspondencia_exata_melhorada
    ALOJAMENTOS: 2 |  Santo António (Nova Freguesia)

  'rua barata salgueiro'
    → 'rua barata salgueiro'
    SCORE: 100.0% | SIMILARIDADE: 1.000 | Método: correspondencia_exata_melhorada
    ALOJAMENTOS: 1 |  Santo António (Nova Freguesia)

  'rua salitre'
    → 'rua do salitre'
    SCORE: 98.0% | SIMILARIDADE: 1.000 | Método: correspondencia_exata_melhorada
    ALOJAMENTOS: 39 |  Santo António (Nova Freguesia)

  'rua são josé'
    → 'rua de são josé'
    SCORE: 98.0% | SIMILARIDADE: 1.000 | Método: correspondencia_exata_melh

In [21]:

dict_correspondencias = {}
if resultados:
    for resultado in resultados:
        rua_original = resultado['rua_alojamento']
        dict_correspondencias[rua_original] = {
            'endereco_limpo': resultado['rua_oficial'],
            'score': resultado['score_fuzzy'],
            'metodo': resultado['metodo'],
            'similaridade_contextual': resultado['similaridade_contextual']
        }

def determinar_sugestao(score, similaridade_contextual, metodo=None):
    if score is None or pd.isna(score) or similaridade_contextual is None or pd.isna(similaridade_contextual):
        return 'sem_correspondencia'
    
    if metodo == 'correspondencia_exata_melhorada' and score >= 97:
        return 'done'
    
    if score >= 98:
        return 'done'
    
    if score >= 95 and similaridade_contextual >= 0.4:
        return 'done'
    elif score >= 90 and similaridade_contextual >= 0.4:
        return 'done'
    elif score >= 85 and similaridade_contextual >= 0.6:
        return 'done'
    elif score >= 80:
        return 'precisa_analise_manual'
    else:
        return 'precisa_analise_manual'

df_smm_completo = pd.read_csv('smm.csv')

if 'sugestao' not in df_smm_completo.columns:
    df_smm_completo['sugestao'] = 'sem_correspondencia'
if 'endereco_limpo' not in df_smm_completo.columns:
    df_smm_completo['endereco_limpo'] = ''

def aplicar_resultado_fuzzy(row):
    if row['Freguesia'] != 'Santo António':
        return row['sugestao'], row['endereco_limpo']
    
    rua_padronizada = row['RuaPadronizada']
    
    if pd.isna(rua_padronizada) or detectar_dados_malformados(rua_padronizada):
        return 'sem_correspondencia', ''
    
    if rua_padronizada in dict_correspondencias:
        resultado = dict_correspondencias[rua_padronizada]
        sugestao = determinar_sugestao(
            resultado['score'], 
            resultado['similaridade_contextual'],
            resultado['metodo']
        )
        return sugestao, resultado['endereco_limpo']
    else:
        return 'sem_correspondencia', ''


df_smm_completo[['sugestao', 'endereco_limpo']] = df_smm_completo.apply(
    lambda row: pd.Series(aplicar_resultado_fuzzy(row)), axis=1
)

print("Total:", len(df_smm_completo[df_smm_completo['Freguesia'] == 'Santo António']))
print("Done:", len(df_smm_completo[(df_smm_completo['Freguesia'] == 'Santo António') & (df_smm_completo['sugestao'] == 'done')]))
print("Manual analysis:", len(df_smm_completo[(df_smm_completo['Freguesia'] == 'Santo António') & (df_smm_completo['sugestao'] == 'precisa_analise_manual')]))
print("Streets that do not correspond to the neighborhood:", len(df_smm_completo[(df_smm_completo['Freguesia'] == 'Santo António') & (df_smm_completo['sugestao'] == 'sem_correspondencia')]))


#df_smm_completo.to_csv('finalfreguesia.csv', index=False, encoding='utf-8')


Total: 1531
Done: 1460
Manual analysis: 25
Streets that do not correspond to the neighborhood: 46


# Cleaning the adresses

In [25]:
smm = pd.read_csv('smm.csv')



In [ ]:
mask = smm['RuaPadronizada'] == 'rua mae de agua'
smm.loc[mask, 'RuaMatch'] = 'rua da mãe dágua'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm["RuaPadronizada"] == "rua mãe d'água"
smm.loc[mask, "RuaMatch"] = "rua da mãe dágua"
smm.loc[mask, "LimpezaDados"] = "manual"

mask = smm['RuaPadronizada'] == 'travessa fala-só'
smm.loc[mask, 'RuaMatch'] = 'travessa do falasó'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'travessa travessa de santa marta'
smm.loc[mask, 'RuaMatch'] = 'travessa de santa marta'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'rua rua prior coutinho'
smm.loc[mask, 'RuaMatch'] = 'rua do prior coutinho'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'rua rua do passadiço'
smm.loc[mask, 'RuaMatch'] = 'rua do passadiço'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'rua do passadico'
smm.loc[mask, 'RuaMatch'] = 'rua do passadiço'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'rua rua do passadiço'
smm.loc[mask, 'RuaMatch'] = 'rua do passadiço'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'rua rua do carrião'
smm.loc[mask, 'RuaMatch'] = 'rua do carrião'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'canada rua do cardal de são josé'
smm.loc[mask, 'RuaMatch'] = 'rua do cardal de são josé'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'rua rua duque de palmela'
smm.loc[mask, 'RuaMatch'] = 'rua duque de palmela'
smm.loc[mask, 'LimpezaDados'] = 'manual'

smm.loc[9776, 'RuaMatch'] = 'rua ferreira lapa'
smm.loc[9776, 'LimpezaDados'] = 'manual'

smm.loc[12260, 'RuaMatch'] = 'largo das olarias'
smm.loc[12260, 'LimpezaDados'] = 'freguesia'

mask = smm['RuaPadronizada'] == 'rua rua monte oliveti'
smm.loc[mask, 'RuaMatch'] = 'rua do monte olivete'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'rua rua monte olivete nº19a a'
smm.loc[mask, 'RuaMatch'] = 'rua do monte olivete'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'rua rua do monte olivete'
smm.loc[mask, 'RuaMatch'] = 'rua do monte olivete'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'rua rua júlio césar machado rua júlio césar machado'
smm.loc[mask, 'RuaMatch'] = 'rua júlio césar machado'
smm.loc[mask, 'LimpezaDados'] = 'manual'

smm.loc[9893, 'RuaMatch'] = 'rua joaquim antónio de aguiar'
smm.loc[9893, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'travessa travessa do noronha'
smm.loc[mask, 'RuaMatch'] = 'travessa do noronha'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'avenida da liberdade/rua rodrigo sampaio'
smm.loc[mask, 'RuaMatch'] = 'avenida da liberdade'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'calçada avenida da liberdade'
smm.loc[mask, 'RuaMatch'] = 'avenida da liberdade'
smm.loc[mask, 'LimpezaDados'] = 'manual'

mask = smm['RuaPadronizada'] == 'rua da gloria'
smm.loc[mask, 'RuaMatch'] = 'rua da glória'
smm.loc[mask, 'LimpezaDados'] = 'manual'

smm.loc[15274, 'RuaMatch'] = 'rua da palma'
smm.loc[15274, 'LimpezaDados'] = 'freguesia'

mask = smm['RuaPadronizada'] == 'rua da caridade tornejando para a do cardall a s. josé'
smm.loc[mask, 'RuaMatch'] = 'rua da caridade'
smm.loc[mask, 'LimpezaDados'] = 'manual'

smm.loc[18310, 'LimpezaDados'] = 'cuidado'


In [74]:
smm[smm['RuaPadronizada'] == 'rua da caridade tornejando a do cardal a são josé']

,NrRegisto,DataRegisto,NomeAlojamento,Modalidade,NrCamas,NrUtentes,Endereco,CodPostal,Localidade,Freguesia,Concelho,Distrito,TitulardaExploracao,CleanSafe,Latitude,Longitude,Geocode_Status,CP,Rua,rua_padronizada,endereco_limpo,score,metodo,sugestao
16754,87357/AL,2018-10-20,Rua da Caridade 33,Apartamento,2.0,4,"Rua da Caridade tornejando a do Cardal A São José, 33",1150-090,Lisboa,Lisboa,Lisboa,Santo António,"{'Tipo': 'Pessoa coletiva', 'Nome': 'REACHMOON - CONSTRU...",False,38.721122,-9.143834,SUCCESS,NaN,Rua da Caridade tornejando a do Cardal A São José,rua da caridade tornejando a do cardal a são josé,rua da caridade,NaN,NaN,manual


In [43]:

def padronizar_sugestao(df):
    return df['LimpezaDados'].astype(str).str.strip().str.lower()

valores_para_corrigir = {'sem_correspondencia', 'precisa_analise_manual', '', 'nan', 'none'}
valores_ja_alterados = {'manual', 'freguesia', 'done'}

df_para_corrigir = smm.copy()
df_para_corrigir['indice_original'] = df_para_corrigir.index
df_para_corrigir['sugestao'] = padronizar_sugestao(df_para_corrigir)
df_para_corrigir = df_para_corrigir[df_para_corrigir['LimpezaDados'].isin(valores_para_corrigir)]

colunas_disponveis = []
colunas_desejadas = ['NrRegisto', 'CodPostal', 'NomeAlojamento', 'Endereco', 'RuaPadronizada', 'RuaMatch', 'LimpezaDados']

for col in colunas_desejadas:
    if col in df_para_corrigir.columns:
        colunas_disponveis.append(col)
    else:
        print(f" '{col}' não encontradada dataset")

df_para_corrigir = df_para_corrigir[colunas_disponveis]

print("ENDEREÇOS QUE PRECISAM SER CORRIGIDOS")
print(f"Total: {len(df_para_corrigir)} endereços\n")
display(df_para_corrigir)

smm['LimpezaDados'].value_counts()


ENDEREÇOS QUE PRECISAM SER CORRIGIDOS
Total: 0 endereços



,NrRegisto,CodPostal,NomeAlojamento,Endereco,RuaPadronizada,RuaMatch,LimpezaDados


LimpezaDados
done       17552
manual      1059
cuidado       49
Name: count, dtype: int64

In [80]:
smm.to_csv('smm.csv', index=False)
